# TP3: Problemas NP-Completos

### ¿Hitting-Set Problem está en NP?

In [1]:
#Verificamos la solucion polinomialmente
def verificar_solucion(B, C):
    satisface_Bi = [False] * len(B)

    for c in C:                      # O(#C) 
        for i in range(len(B)):
            if c in B[i]:            # O(#B[i])
                satisface_Bi[i] = True

    return all(satisface_Bi)      # O(#B)         

#### Funciones auxiliares


In [2]:
# Recibe el path con el archivo a leer.
# Leemos y devolvemos dos arrays. 
# - El primero corresponde a un array de listas, donde cada lista posee los jugadores que piden los diferentes medios y periodistas.
# - El segundo array posee todos los jugadores que aparecen en el array anterior, siendo los jugadores que Scaloni puede elegir, su preselección.
# Este algoritmo, devuelve una preselección sin orden.
def obtener_preseleccion_y_pedidos_de_la_prensa(path):
    with open(path, 'r') as archivo:
        lineas = archivo.readlines()

    pedidos_de_la_prensa = []
    preseleccion = set()

    for linea in lineas:
        preseleccion_linea = linea.strip().split(',')
        pedidos_de_la_prensa.append(preseleccion_linea)
        preseleccion.update(set(preseleccion_linea))

    return pedidos_de_la_prensa, list(preseleccion)


In [3]:
#Paths de los archivos a utilizar
path_5 = 'sets de la catedra/5.txt'
path_7 = 'sets de la catedra/7.txt'
path_10_pocos = 'sets de la catedra/10_pocos.txt'
path_10_todos = 'sets de la catedra/10_todos.txt'
path_10_varios = 'sets de la catedra/10_varios.txt'
path_15 = 'sets de la catedra/15.txt'
path_20 = 'sets de la catedra/20.txt'
path_75 = 'sets de la catedra/75.txt'
path_100 = 'sets de la catedra/100.txt'
path_200 = 'sets de la catedra/200.txt'
resultados_sets_catedra = 'sets de la catedra/Resultados Esperados.txt'

In [45]:
# Me guardo los resultados esperados, dados por la cátedra
class Resultado:
    def __init__(self, nombre_archivo, cantidad_minima, jugadores):
        self.nombre_archivo = nombre_archivo
        self.cantidad_minima = cantidad_minima
        self.jugadores = jugadores

def obtener_resultados_esperados(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        contenido = archivo.read() 
    resultados = []

    # Separa el contenido en bloques
    bloques = contenido.split('\n\n')
    
    # Itera a través de cada bloque y busca "Ganancia maxima" y "Plan de entrenamiento" dentro de cada uno
    for bloque in bloques:
        nombre_archivo = ''
        cantidad_minima = 0
        jugadores = []

        lineas = bloque.split('\n')
        for linea in lineas:
            if ".txt" in linea:
                nombre_archivo = linea
            if 'Cantidad' in linea:
                cantidad_minima = int(linea.split(":")[1].split("(")[0].strip())
                jugadores_linea = linea.split("(")[1].split(")")[0]
                jugadores = [jugador.strip() for jugador in jugadores_linea.split(",")]
                resultados.append(Resultado(nombre_archivo, cantidad_minima, jugadores))

    return resultados


### Función verificadora

In [5]:
#Tomamos como base la verificación del Hitting-Set Problem. Y lo ajustamos para nuestro problema.
def verificar_solucion_para_Scaloni(B, C):
    if len(C) == 0:
        return False
    
    satisface_Bi = [False] * len(B)

    for c in C:                      # O(#C) 
        for i in range(len(B)):
            if c in B[i]:            # O(#B[i])
                satisface_Bi[i] = True

    return all(satisface_Bi)      # O(#B) 

### Solución por Backtracking


In [6]:
def actualizar_solucion(solucion_actual, solucion):
    if len(solucion_actual) < len(solucion) or len(solucion) == 0:
        solucion[:] = solucion_actual


def conjunto_minimo_para_Scaloni(preseleccion, solucion_actual, pedidos_de_prensa, indice, solucion):
    cantidad_de_jugadores = len(solucion_actual)
    max_jugadores = 16

    if(cantidad_de_jugadores >= len(pedidos_de_prensa) or cantidad_de_jugadores >= max_jugadores):
        return 
    
    for i in range(indice, (len(preseleccion)-1)):
        solucion_actual.append(preseleccion[(i)])
        if verificar_solucion_para_Scaloni(pedidos_de_prensa, solucion_actual):
            actualizar_solucion(solucion_actual, solucion)
        conjunto_minimo_para_Scaloni(preseleccion, solucion_actual, pedidos_de_prensa, i + 1, solucion)
        solucion_actual.remove(solucion_actual[len(solucion_actual) - 1])



def solucion_para_Scaloni(preseleccion, pedidos_de_prensa):
    solucion = []
    solucion_actual = []
    conjunto_minimo_para_Scaloni(preseleccion, solucion_actual, pedidos_de_prensa, 0, solucion)
    return solucion


# Ejemplo de uso
archivo_path = path_7
pedidos_de_la_prensa, preseleccion = obtener_preseleccion_y_pedidos_de_la_prensa(archivo_path)
solucion = solucion_para_Scaloni(preseleccion, pedidos_de_la_prensa)
print(solucion)

['Mauro Zarate', 'Pezzella']


#### Chequeo solución 

##### 1 - Usando sets de la cátedra

In [7]:
# correr pip install tabulate en consola para poder usar esta libreria 
from tabulate import tabulate

In [32]:
pedidos_de_la_prensa_5, preseleccion_5 = obtener_preseleccion_y_pedidos_de_la_prensa(path_5)
solucion_5 = solucion_para_Scaloni(preseleccion_5, pedidos_de_la_prensa_5)
pedidos_de_la_prensa_7, preseleccion_7 = obtener_preseleccion_y_pedidos_de_la_prensa(path_7)
solucion_7 = solucion_para_Scaloni(preseleccion_7, pedidos_de_la_prensa_7)
pedidos_de_la_prensa_10_pocos, preseleccion_10_pocos = obtener_preseleccion_y_pedidos_de_la_prensa(path_10_pocos)
solucion_10_pocos = solucion_para_Scaloni(preseleccion_10_pocos, pedidos_de_la_prensa_10_pocos)
pedidos_de_la_prensa_10_varios, preseleccion_10_varios = obtener_preseleccion_y_pedidos_de_la_prensa(path_10_varios)
solucion_10_varios = solucion_para_Scaloni(preseleccion_10_varios, pedidos_de_la_prensa_10_varios)
pedidos_de_la_prensa_10_todos, preseleccion_10_todos = obtener_preseleccion_y_pedidos_de_la_prensa(path_10_todos)
solucion_10_todos = solucion_para_Scaloni(preseleccion_10_todos, pedidos_de_la_prensa_10_todos)


In [50]:
resultados_esperados = obtener_resultados_esperados(resultados_sets_catedra)[:5]
pedidos_de_la_prensa = [pedidos_de_la_prensa_5, pedidos_de_la_prensa_7, pedidos_de_la_prensa_10_pocos,\
                        pedidos_de_la_prensa_10_varios, pedidos_de_la_prensa_10_todos]
resultados_obtenidos = [solucion_5, solucion_7, solucion_10_pocos,solucion_10_varios, solucion_10_todos]

In [52]:
# Imprime los resultados obtenidos y los esperados en una tabla
resultados = []
i = 0
for resultado_esperado in resultados_esperados:
  es_correcto = "Sí" if resultado_esperado.cantidad_minima == len(resultados_obtenidos[i]) and verificar_solucion_para_Scaloni(pedidos_de_la_prensa[i], resultados_obtenidos[i]) else "No"
  resultados.append([resultado_esperado.nombre_archivo, resultado_esperado.cantidad_minima, len(resultados_obtenidos[i]), es_correcto]) 
  i += 1
print(tabulate(resultados, headers=['Archivo', 'Cantidad mínima esperada', 'Cantidad mínima obtenida', 'Se obtuvo el resultado esperado?']))

Archivo          Cantidad mínima esperada    Cantidad mínima obtenida  Se obtuvo el resultado esperado?
-------------  --------------------------  --------------------------  ----------------------------------
5.txt                                   2                           2  Sí
7.txt                                   2                           2  Sí
10_pocos.txt                            3                           3  Sí
10_varios.txt                           6                           6  Sí
10_todos.txt                           10                          10  Sí
